# Chapter 3: Toy IR

Now that we're familiar with our language and the target instruction set, let's begin
compiling step-by-step.

## Introduction: Multi-Level Intermediate Representation

xDSL leverages the MLIR representation of a program. MLIR specified a text format of
this representation, which is useful for debugging and interoperation. For example,
all the text in this format you'll see in this tutorial can be executed with the 
Toy language as compiled in the (MLIR Toy Tutorial)[https://mlir.llvm.org/docs/Tutorials/Toy/].

Let's take a quick look at the MLIR IR representation of our example program:

In [1]:
from compiler import parse_toy, print_op

example = """
def main() {
  var a<2, 3> = [[1, 2, 3], [4, 5, 6]];
  var b<3, 2> = [1, 2, 3, 4, 5, 6];
  var c<2, 3> = b;
  var d = a + c;
  print(d);
}
"""

toy_0 = parse_toy(example)
print_op(toy_0)
print()

"builtin.module"() ({
  "toy.func"() ({
    %0 = "toy.constant"() {"value" = dense<[[1, 2, 3], [4, 5, 6]]> : tensor<2x3xi32>} : () -> tensor<2x3xi32>
    %1 = "toy.reshape"(%0) : (tensor<2x3xi32>) -> tensor<2x3xi32>
    %2 = "toy.constant"() {"value" = dense<[1, 2, 3, 4, 5, 6]> : tensor<6xi32>} : () -> tensor<6xi32>
    %3 = "toy.reshape"(%2) : (tensor<6xi32>) -> tensor<3x2xi32>
    %4 = "toy.reshape"(%3) : (tensor<3x2xi32>) -> tensor<2x3xi32>
    %5 = "toy.add"(%1, %4) : (tensor<2x3xi32>, tensor<2x3xi32>) -> tensor<2x3xi32>
    "toy.print"(%5) : (tensor<2x3xi32>) -> ()
    "toy.return"() : () -> ()
  }) {"sym_name" = "main", "function_type" = () -> ()} : () -> ()
}) : () -> ()



As you might have noticed, some parts look very similar to the Toy program above, and some
things are added in. Let's look at the structure of an operation in the MLIR output before
taking a close look at exactly what's going on.

## MLIR Syntax In Detail

MLIR is designed to be a completely extensible infrastructure; there is no
closed set of attributes (think: constant metadata), operations, or types. MLIR
supports this extensibility with the concept of Dialects. Dialects provide a grouping 
mechanism for abstraction under a unique `namespace`.

In MLIR, `Operations` are the core unit of abstraction and computation, similar in many 
ways to LLVM instructions. Operations can have application-specific semantics and can be 
used to represent all of the core IR structures in LLVM: instructions, globals (like 
functions), modules, etc.

Here is the MLIR assembly for the Toy `transpose` operations:

```mlir
%t_tensor = "toy.transpose"(%tensor): (tensor<2x3xi32>) -> tensor<3x2xi32>
```

Let's break down the anatomy of this MLIR operation:

-   `%t_tensor`

    *   The name given to the result defined by this operation (which includes
        [a prefixed sigil to avoid collisions](../../LangRef.md/#identifiers-and-keywords)).
        An operation may define zero or more results (in the context of Toy, we
        will limit ourselves to single-result operations), which are SSA values.
        The name is used during parsing but is not persistent (e.g., it is not
        tracked in the in-memory representation of the SSA value).

-   `"toy.transpose"`

    *   The name of the operation. It is expected to be a unique string, with
        the namespace of the dialect prefixed before the "`.`". This can be read
        as the `transpose` operation in the `toy` dialect.

-   `(%tensor)`

    *   A list of zero or more input operands (or arguments), which are SSA
        values defined by other operations or referring to block arguments.


-   `(tensor<2x3xf64>) -> tensor<3x2xf64>`

    *   This refers to the type of the operation in a functional form, spelling
        the types of the arguments in parentheses and the type of the return
        values afterward.


Shown here is the general form of an operation. As described above,
the set of operations in MLIR is extensible. Operations are modeled
using a small set of concepts, enabling operations to be reasoned
about and manipulated generically. These concepts are:

-   A name for the operation.
-   A list of SSA operand values.
-   A list of attributes.
-   A list of types for result values.
-   A source location for debugging purposes.
-   A list of successors blocks (for branches, mostly).
-   A list of regions (for structural operations like functions).

## Defining Toy Operations

Now that we have a `Toy` dialect, we can start defining the operations. This
will allow for providing semantic information that the rest of the system can
hook into. As an example, let's walk through the creation of a `toy.constant`
operation. This operation will represent a constant value in the Toy language.

```mlir
 %4 = "toy.constant"() {value = dense<[[1, 2, 3], [4, 5, 6]]> : tensor<2x3xi32>} : () -> tensor<2x3xi32>
```

This operation takes zero operands, a dense elements attribute named `value` 
to represent the constant value, and returns a single result of RankedTensorType. 
Let's take a look at the full definition and step through it in detail.


In [2]:
from __future__ import annotations

from typing import Annotated, TypeAlias

# The xdsl.ir module implements the things we've mentioned in this chapter,
# especially the equivalents of MLIR concepts.
from xdsl.ir import Dialect, SSAValue, Attribute, Operation, OpResult

# The builtin dialect is a collection of Operations, and Attributes that are expected
# to be useful for most compilers, such as floating-point numbers, integers,
# arrays, tensors, and more.
from xdsl.dialects.builtin import TensorType, IntegerType, i32, DenseIntOrFPElementsAttr

# xdsl.irdl provides a declarative Python API for Operation definitions
from xdsl.irdl import OpAttr, irdl_op_definition



TensorTypeI32: TypeAlias = TensorType[IntegerType]


# A decorator to help implement some methods required by xDSL
@irdl_op_definition
class ConstantOp(Operation):
    """
    Constant operation turns a literal into an SSA value. The data is attached
    to the operation as an attribute. For example:

    ```mlir
      %0 = "toy.constant"() {"value" = dense<[[1, 2, 3], [4, 5, 6]]> : tensor<2x3xi32>} : () -> tensor<2x3xi32>
    ```
    """
    # Every operation has a name. The format is `dialect_name`.`operation_name`
    name: str = "toy.constant"

    # Attributes are defined using OpAttr, and can specify a type constraint on the attribute
    value: OpAttr[DenseIntOrFPElementsAttr]

    # The result type annotation uses `Annotated`, a type in the `typing` module that
    # allows for runtime annotation of types with arbitrary values. xDSL leverages
    # this annotation to populate a `verify()` method that will signal if there is
    # a type mismatch during construction.
    res: Annotated[OpResult, TensorTypeI32]

    # Operations can provide helper constructors for ease of use
    @staticmethod
    def from_list(data: list[int], shape: list[int]) -> ConstantOp:
        value = DenseIntOrFPElementsAttr.tensor_from_list(data, i32, shape)
        return ConstantOp.create(attributes={'value': value}, result_types=[value.type])


from compiler import print_op

print_op(ConstantOp.from_list([1, 2, 3, 4, 5, 6], [2, 3]))

%0 = "toy.constant"() {"value" = dense<[[1, 2, 3], [4, 5, 6]]> : tensor<2x3xi32>} : () -> tensor<2x3xi32>


## Another Look at the Generated Toy IR

Let's take another look at the generated IR:

``` MLIR
"builtin.module"() ({
  "toy.func"() ({
    %0 = "toy.constant"() {"value" = dense<[[1, 2, 3], [4, 5, 6]]> : tensor<2x3xi32>} : () -> tensor<2x3xi32>
    %1 = "toy.reshape"(%0) : (tensor<2x3xi32>) -> tensor<2x3xi32>
    %2 = "toy.constant"() {"value" = dense<[1, 2, 3, 4, 5, 6]> : tensor<6xi32>} : () -> tensor<6xi32>
    %3 = "toy.reshape"(%2) : (tensor<6xi32>) -> tensor<3x2xi32>
    %4 = "toy.reshape"(%3) : (tensor<3x2xi32>) -> tensor<2x3xi32>
    %5 = "toy.add"(%1, %4) : (tensor<2x3xi32>, tensor<2x3xi32>) -> tensor<2x3xi32>
    "toy.print"(%5) : (tensor<2x3xi32>) -> ()
    "toy.return"() : () -> ()
  }) {"sym_name" = "main", "function_type" = () -> ()} : () -> ()
}) : () -> ()
```

First thing we see is that the whole program is wrapped in `builtin.module`. Modules
roughly represent a parsed input file. Nested one level deep is a `toy.func`, representing
the `main` function defined in the source. Inside it are a list of instructions.

The next four lines of MLIR ops correspond to the first two lines of the Toy program.

``` MLIR
%0 = "toy.constant"() {"value" = dense<[[1, 2, 3], [4, 5, 6]]> : tensor<2x3xi32>} : () -> tensor<2x3xi32>
%1 = "toy.reshape"(%0) : (tensor<2x3xi32>) -> tensor<2x3xi32>
%2 = "toy.constant"() {"value" = dense<[1, 2, 3, 4, 5, 6]> : tensor<6xi32>} : () -> tensor<6xi32>
%3 = "toy.reshape"(%2) : (tensor<6xi32>) -> tensor<3x2xi32>
```

``` Python
var a<2, 3> = [[1, 2, 3], [4, 5, 6]];
var b<3, 2> = [1, 2, 3, 4, 5, 6];
```

Because the types on the left of the `=` operator might not correspond to the type of the 
literal, the reshape operations are inserted. Most of the time the shapes will match,
and the reshape will be redundant. Let's take a look at how to optimise our code to remove
redundant transposes using the xDSL infrastructure.